In [1]:
from pathlib import Path
from time import sleep 

import numpy as np
import pandas as pd
# import hvplot.pandas

import roboticstoolbox as rtb
import spatialmath as sm

from manipylator.base import VisualRobot
from manipylator import utils

# Load Robot
Below we load a URDF file describing Manny the robot manipulator. A URDF is a textual representation of a robot, defining the parameters we need so we can calculate the forward & reverse kinemtics. 

In [2]:
# path = Path('/workspace/robots/robot.urdf')
# path = Path('/workspace/robots/robot-ee.urdf')
path = Path('/workspace/robots/measured/robot.urdf')

# manny = VisualRobot(path)
manny = VisualRobot(path)

print(manny.model)

[I 05/28/25 20:20:37.886 32] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout
[Genesis] [20:20:44] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [20:20:44] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [20:20:44] [INFO] ╰───────────────────────────────────────────────╯
/opt/conda/lib/python3.11/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
[Genesis] [20:20:44] [INFO] Running on [Intel(R) Core(TM) i7-8665U CPU @ 1.90GHz] with backend gs.vulkan. Device memory: 15.25 GB.
RHI Error: Can not create Vulkan instance
[W 05/28/25 20:20:44.974 32] [misc.py:adaptive_arch_select@758] Arch=[<Arch.vulka

ERobot: measured, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬──────────────────┬───────┬──────────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ link │       link       │ joint │      parent      │                             ETS: parent to link                             │
├──────┼──────────────────┼───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│    0 │ base             │       │ BASE             │ SE3()                                                                       │
│    1 │ carriage_1       │     0 │ base             │ SE3(-7.47e-06, -2.937e-06, 0.037; -180°, -7.914e-41°, -6.361e-15°) ⊕ Rz(q0) │
│    2 │ shoulder_lift    │     1 │ carriage_1       │ SE3(-0.065, -2.469e-06, -0.055; 180°, 90°, 180°) ⊕ Rz(q1)                   │
│    3 │ elbow            │     2 │ shoulder_lift    │ SE3(0.355, -2.338e-06, -4.032e-06; -180°, -3.45e-15°, 3.069e-12°) ⊕ Rz(q2)  │
│ 

# Visualizing Manny
Using the `manny.visualizer.robot` object we can get the properties of it's links. For example, a given link's location in 3D space. Below we find the location of the end effector in when the all joint parameters are set to 0 degrees:

In [3]:
robot = manny.visualizer.robot
ee = robot.get_link('end_effector')

# ee
# robot

# Tracing a Path in R3
Below we calculate a positions in R3 that correspond to a heart shaped curve:

In [4]:
scale = 0.06
x_offset = -0.0885
y_offset = 0.36
height = 0.12

# -0.0885,  0.2954,  0.2595
# shape = np.array([[0, 0, 0],
#                   [0.25, 0, 0],                  
#                   [0.5, 0, 0],                  
#                   [0.75, 0, 0],                  
#                   [1, 0, 0],
#                   [1, 0.25, 0],
#                   [1, 0.5, 0],
#                   [1, 0.75, 0],
#                   [1, 1, 0],
#                   [0.75, 1, 0],
#                   [0.5, 1, 0],
#                   [0.25, 1, 0],
#                   [0, 1, 0],
#                   [0, 0.75, 0],
#                   [0, 0.5, 0],
#                   [0, 0.25, 0],
#                  ])

shape = utils.parametric_heart_1(np.linspace(0, 2*np.pi, 40))


positions = scale * shape
positions[:,0] += x_offset
positions[:,1] += y_offset
positions[:,2] = height

positions[:3]

array([[-0.0885    ,  0.36      ,  0.12      ],
       [-0.08814976,  0.36229084,  0.12      ],
       [-0.0858055 ,  0.36863598,  0.12      ]])

Each row is the (x, y, z) coordinates of a point on the curve. This is useful if we'd like to visualize the expected path of the end effector.

In [5]:
xy = positions[:, :2]
df = pd.DataFrame(xy, columns=['x', 'y'])
df.head()

,x,y
0,-0.088500,0.360000
1,-0.088150,0.362291
2,-0.085805,0.368636
3,-0.079984,0.377559
4,-0.070104,0.386931


In [10]:
# for x, y, z in positions:
#     manny.visualizer.scene.draw_debug_sphere([x,y,z])
manny.visualizer.scene.clear_debug_objects()
manny.visualizer.scene.draw_debug_spheres(positions)



Manually set dofs to sane initial position to make life easier for inverse kinematics

In [11]:
robot.set_dofs_position([0, -0.2, 2.1, 1, 1.57, 0])

manny.visualizer.scene.step()
ee.get_pos()

tensor([-0.0885,  0.3720,  0.2415])

In [12]:
ee.get_quat()

tensor([9.9085e-01, 1.3498e-01, 3.9601e-04, 5.2616e-05])

Similarly, we can calculate the joint parameters of each position:

In [13]:
quat_pointing_down = [1, 0, 0, 0]

initial_pose = robot.inverse_kinematics(
        link = ee,
        pos  = positions[0],
        quat = quat_pointing_down,
        )

robot.set_dofs_position(initial_pose)
manny.visualizer.scene.step()

[W 05/28/25 20:25:52.248 32] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


In [19]:

qs = [robot.inverse_kinematics(
        link = ee,
        pos  = pos,
        quat = quat_pointing_down,
        ) for pos in positions]
qs[:5]

[tensor([ 2.7524e-05, -4.5751e-01,  2.2548e+00,  1.1415e+00,  1.5708e+00,
          3.1546e-05]),
 tensor([ 1.0011e-03, -4.6082e-01,  2.2469e+00,  1.1369e+00,  1.5708e+00,
          1.0046e-03]),
 tensor([ 0.0075, -0.4695,  2.2263,  1.1250,  1.5708,  0.0075]),
 tensor([ 0.0237, -0.4809,  2.1995,  1.1096,  1.5708,  0.0237]),
 tensor([ 0.0478, -0.4931,  2.1711,  1.0935,  1.5708,  0.0478])]

In [26]:
robot.set_dofs_position(qs[21])
manny.visualizer.scene.step()
print(f'position: {ee.get_pos()}\ndirection: {ee.get_quat()}')


position: tensor([-0.0897,  0.2418,  0.1199])
direction: tensor([ 1.0000e+00, -9.3728e-06,  0.0000e+00,  1.9372e-07])


In [27]:
df = pd.DataFrame(qs, columns=['q1', 'q2', 'q3', 'q4', 'q5', 'q6']).astype(float)
df['q6'] = 0
df['q5'] = 1.57
df.head()
df.to_csv('trajectory_qs.csv', index=False, float_format='%.3f')

And to visualize the poses Manny would take, we can animate the path:

In [28]:
# camera = manny.visualizer.camera
# camera.start_recording()

for q in qs+qs[:1]:
    robot.set_dofs_position(q)
    # print(robot.get_dofs_position())
    manny.visualizer.scene.step()
    # camera.render()
    
    sleep(0.5)

# camera.stop_recording(save_to_filename='video.mp4', fps=20)

In [ ]:
# Reset to zero pose
robot.set_dofs_position([0]*6)
manny.visualizer.scene.step()
